In [1]:
import sqlite3
connection = sqlite3.connect("inaturalist-open-data-20220127.sq3db")

In [2]:
print(connection.total_changes)

0


In [3]:
cursor = connection.cursor()
rows = cursor.execute("SELECT * FROM photos LIMIT 3").fetchall()
print(rows)

[('photo_uuid', 'photo_id', 'observation_uuid', 'observer_id', 'extension', 'license', 'width', 'height', 'position'), ('a291d420-3ca6-4f94-b955-6c936a9b596d', 35926, '526ababd-aff2-4ce8-b111-d8013b9d8b42', 505, 'jpg', 'CC-BY', 2048, 1680, 0), ('c6d7769c-0390-42cc-8012-874cd49ea180', 35925, '526ababd-aff2-4ce8-b111-d8013b9d8b42', 505, 'jpg', 'CC-BY', 2048, 1863, 1)]


In [4]:
target_photo_id = 31539
rows = cursor.execute(
    "SELECT photo_id, license FROM photos WHERE photo_id = ?", 
    (target_photo_id, ),
).fetchall()
print(rows)

[(31539, 'CC-BY')]


In [5]:
rows = cursor.execute(
    "SELECT COUNT(DISTINCT taxon_id), COUNT(DISTINCT name)"
    "FROM taxa "
    "LIMIT 10 "
).fetchall()

print(rows)

[(1311566, 1295775)]


In [6]:
rows = cursor.execute(
    "SELECT COUNT(DISTINCT ancestry)"
    "FROM taxa "
    "LIMIT 10 "
).fetchall()

print(rows)

[(199006,)]


In [7]:
# query all insects 
rows = cursor.execute(
    "SELECT *"
    "FROM taxa "
    "WHERE name = 'Insecta' "
    "LIMIT 10 "
).fetchall()

print(rows)

[(47158, '48460/1/47120/372739', 50.0, 'class', 'Insecta', 'true')]


In [8]:
# query all hexapods
rows = cursor.execute(
    "SELECT *"
    "FROM taxa "
    "WHERE taxon_id = 372739 "
    "LIMIT 10 "
).fetchall()

print(rows)

[(372739, '48460/1/47120', 57.0, 'subphylum', 'Hexapoda', 'true')]


In [9]:
# query all insects at insect lvl
rows = cursor.execute(
    "SELECT name, rank "
    "FROM taxa "
    "WHERE ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND rank = 'species' "
    "LIMIT 10 "
).fetchall()

print(rows)

[('Crabro peltarius', 'species'), ('Halyzia hauseri', 'species'), ('Gonitis privata', 'species'), ('Pulvinaria regalis', 'species'), ('Diplotaxis academia', 'species'), ('Polistes dorsalis', 'species'), ('Akis algeriana', 'species'), ('Crabro diacanthus', 'species'), ('Junonia almana', 'species'), ('Ctenophorema balneare', 'species')]


In [10]:
# distinct number of insects
rows = cursor.execute(
    "SELECT COUNT(DISTINCT taxon_id) "
    "FROM taxa "
    "WHERE ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND rank = 'species' "
    "LIMIT 10 "
).fetchall()

print(rows)

[(403111,)]


In [11]:
rows = cursor.execute(
    "SELECT name, taxon_id "
    "FROM taxa "
    "WHERE ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND rank = 'species' "
    "GROUP BY taxon_id "
    "HAVING COUNT(name) > 1 "
    "LIMIT 10 "
).fetchall()

print(rows)

[]


In [12]:
# test of 2 distinct taxon_ids for the same name
rows = cursor.execute(
    "SELECT name, taxon_id, ancestry "
    "FROM taxa "
    "WHERE ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND rank = 'species' "
    "AND name = 'Acleris logiana' "
    "LIMIT 10 "
).fetchall()

print(rows)

[('Acleris logiana', 212338, '48460/1/47120/372739/47158/184884/47157/47156/47155/124615/199539/124620'), ('Acleris logiana', 1127186, '48460/1/47120/372739/47158/184884/47157/47156/47155/124615/199539/124620')]


In [13]:
# test of 2 distinct taxon_ids for the same name
rows = cursor.execute(
    "SELECT name, taxon_id, ancestry "
    "FROM taxa "
    "WHERE ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND rank = 'species' "
    "AND name = 'Aedes aloponotum' "
    "LIMIT 10 "
).fetchall()

print(rows)

[('Aedes aloponotum', 458895, '48460/1/47120/372739/47158/184884/47822/154259/153429/513826/52134/357355/505234/62989'), ('Aedes aloponotum', 1112054, '48460/1/47120/372739/47158/184884/47822/154259/153429/513826/52134/357355/505234/62989/1111823')]


In [ ]:
rows = cursor.execute(
    "WITH urls AS (SELECT 'http://inaturalist-open-data.s3.amazonaws.com/photos/' AS photo_url) "
    "SELECT B.*, U.photo_url || A.photo_id || '/' || 'large.' || A.extension AS photo_url_large "
    "FROM urls U "
    "CROSS JOIN photos A "
    "JOIN observers B "
    "ON A.observer_id = B.observer_id "
    "JOIN observations C "
    "ON A.observation_uuid = C.observation_uuid "
    "LEFT JOIN taxa D "
    "ON C.taxon_id = D.taxon_id "
    "WHERE D.ancestry LIKE '48460/1/47120/372739/47158/%' "
    "AND D.rank = 'species' "
    "ORDER BY A.photo_id "
    "LIMIT 10 "
).fetchall()
    
print(rows)